In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import copy

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import (StratifiedKFold, train_test_split, 
                                     RandomizedSearchCV, GridSearchCV)
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

from sklearn import pipeline

import optuna
from functools import partial

In [ ]:
train = pd.read_csv('../input/amazon-employee-access-challenge/train.csv')
test = pd.read_csv('../input/amazon-employee-access-challenge/test.csv')

In [ ]:
train

In [ ]:
train.dtypes

In [ ]:
sns.countplot(train['ACTION'])

In [ ]:
train['RESOURCE'].value_counts().sort_values(ascending=False)

In [ ]:
for i in train.columns:
    Mean_encoded_subject = train.groupby([i])['ACTION'].mean().to_dict() 
    train[i] =  train[i].map(Mean_encoded_subject)
    
    if i!='ACTION':
        test[i] =  test[i].map(Mean_encoded_subject)
    
test

In [ ]:
train['ACTION']=train['ACTION'].astype('float64')

In [ ]:
X=train.drop(columns='ACTION').to_numpy()
y=train['ACTION'].to_numpy()

In [ ]:
train = train.astype({"ACTION":'int64'})

In [ ]:
train.dtypes

In [ ]:
sns.pairplot(data=train, kind='scatter')

In [ ]:
train_corr=train.corr()
train_corr['ACTION'].sort_values(ascending=False)

In [ ]:
train=train.drop(columns='ROLE_TITLE')
test=test.drop(columns='ROLE_TITLE')

In [ ]:
sns.relplot(data=train, x="ROLE_FAMILY", y="RESOURCE", hue="ACTION")

In [ ]:
train.columns

In [ ]:
#train = train.astype({"ACTION":'O'})
sns.jointplot(data=train, x="ROLE_FAMILY_DESC", y="RESOURCE", kind="kde")

In [ ]:
sns.jointplot(x='MGR_ID', y='RESOURCE', data=train, kind='kde')

In [ ]:
train

In [ ]:
corr_matrix=train.corr()
corr_matrix['ACTION']

In [ ]:
sns.pairplot(data=train, kind='scatter')

In [ ]:
train['ACTION'].value_counts()

In [ ]:
sns.relplot(data=train,
            x="RESOURCE", 
            y="MGR_ID",
            hue="ROLE_FAMILY",
            #size="ROLE_FAMILY",
            style="ACTION",
            kind="scatter")

In [ ]:
l_enc = LabelEncoder()
X=train.drop(columns='ACTION').to_numpy()
y=train['ACTION'].to_numpy()
y = l_enc.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, stratify=y)

trained= pd.DataFrame(X_train)
trained.columns= train.columns[1:]

def jittered(arr):
    stdev = .01 * (max(arr) - min(arr))
    jit = arr + np.random.randn(len(arr)) * stdev
    #transformed=stats.boxcox(jit +abs(np.min(jit))+0.1)[0]
    
    return jit

for i in train.columns[1:]:
    trained[i] = jittered(trained[i])
    
X_train=trained.to_numpy()
trained

In [ ]:
xgb=XGBClassifier(subsample=0.1)

classifier= pipeline.Pipeline([("xgb",xgb)])

param_grid={
    "xgb__n_estimators":np.arange(100,1500,100),
    "xgb__max_depth": np.arange(3,11,2),
    "xgb__learning_rate": np.arange(0.001,0.1)
}

random_search= RandomizedSearchCV(estimator=classifier,
                         param_distributions=param_grid,
                         scoring='roc_auc',
                         verbose=10,
                         n_iter=100,
                         n_jobs=-1,
                         cv=5)

random_search.fit(X_train,y_train)

print(random_search.best_score_, random_search.best_estimator_.get_params())

In [ ]:
test=test.drop(columns='id')
X=test.to_numpy()

In [ ]:
model=XGBClassifier(n_estimators=1400,max_depth= 7,
                        learning_rate=0.001, subsample=0.1)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
fold_auc = roc_auc_score(y_test, y_pred)
fold_auc

In [ ]:
predictions= model.predict_proba(X)

In [ ]:
sub=pd.read_csv("../input/amazon-employee-access-challenge/sampleSubmission.csv")
sub.Id=np.arange(1,len(test)+1)
sub.Action=predictions[:,1]

sub

In [ ]:
sub.to_csv('amazon_xgboost_submission.csv', index=False, header=True)
sub.head()